In [2]:
import os
from pathlib import Path
import sys
sys.path.extend(['../../', '../../base'])
import re
import imageio
import cv2
import matplotlib.pyplot as plt
from base.utils.annotation_converter import AnnotationConverter
from base.utils.aida_annotation import AIDAnnotation

In [3]:
converter = AnnotationConverter(min_contour_area=40000)
aida_ann = AIDAnnotation('17_A047-4463_153D+-+2017-05-11+09.40.22.ndpi', 'test', ['epithelium', 'lumen', 'background'])

    tile_dir_path = '/Volumes/A-CH-EXDISK/Projects/Dataset/train/17_A047-4463_153D+-+2017-05-11+09.40.22_TissueTrain_(1.00,30774,15012,3897,4556)/tiles'
tile_paths = list(str(path) for path in Path(tile_dir_path).iterdir() if '_mask_' in str(path))

In [ ]:
# extract offset coords from tile name
coords_pattern = '\((\w\.\w{1,3}),(\w{1,6}),(\w{1,6}),(\w{1,6}),(\w{1,6})\)_mask_(\w{1,6}),(\w{1,6})'
for tile_path in tile_paths[0: 3]:  # test on subsets of the tiles
    tile = imageio.imread(tile_path)
    coords_info = re.search(coords_pattern, Path(tile_path).name).groups()  # tuple with all matched groups
    downsample = float(coords_info[0])  # downsample is a float
    area_x, area_y, area_w, area_h, tile_x, tile_y = tuple(int(num) for num in coords_info[1:])
    x_offset = area_x + tile_x
    y_offset = area_y + tile_y
    contours, labels, boxes = converter.mask_to_contour(tile, x_offset, y_offset)
    for contour, label, box in zip(contours, labels, boxes):
        aida_ann.add_item(label, 'path', tile_rect=box)
        contour = contour.squeeze().astype(int).tolist()  # deal with extra dim at pos 1
        aida_ann.add_segments_to_last_item(contour)

In [ ]:
aida_ann_dir = '/Users/andreachatrian/Documents/Repositories/AIDA/dist/data/annotations'
aida_ann.merge_overlapping_segments(dissimilarity_thresh=20.0, max_iter=5)
aida_ann.dump_to_json(aida_ann_dir)

# debug on real data
tile_path0 = '/Volumes/A-CH-EXDISK/Projects/Dataset/train/17_A047-4463_153D+-+2017-05-11+09.40.22_TissueTrain_(1.00,30774,15012,3897,4556)/tiles/17_A047-4463_153D+-+2017-05-11+09.40.22_TissueTrain_(1.00,30774,15012,3897,4556)_mask_1,2049.png'
tile_path1 = '/Volumes/A-CH-EXDISK/Projects/Dataset/train/17_A047-4463_153D+-+2017-05-11+09.40.22_TissueTrain_(1.00,30774,15012,3897,4556)/tiles/17_A047-4463_153D+-+2017-05-11+09.40.22_TissueTrain_(1.00,30774,15012,3897,4556)_mask_628,2508.png'
coords_pattern = '\((\w\.\w{1,3}),(\w{1,6}),(\w{1,6}),(\w{1,6}),(\w{1,6})\)_mask_(\w{1,6}),(\w{1,6})'
for tile_path in [tile_path0, tile_path1]:  # test on subsets of the tiles
    tile = imageio.imread(tile_path)
    coords_info = re.search(coords_pattern, Path(tile_path).name).groups()  # tuple with all matched groups
    downsample = float(coords_info[0])  # downsample is a float
    area_x, area_y, area_w, area_h, tile_x, tile_y = tuple(int(num) for num in coords_info[1:])
    x_offset = tile_x
    y_offset = tile_y
    contours, labels, boxes = converter.mask_to_contour(tile, x_offset, y_offset)
    for contour, label, box in zip(contours, labels, boxes):
        aida_ann.add_item(label, 'path', tile_rect=box)
        contour = contour.squeeze().astype(int).tolist()  # deal with extra dim at pos 1
        aida_ann.add_segments_to_last_item(contour)

# Plot contours before merging
large_image = imageio.imread('/Volumes/A-CH-EXDISK/Projects/Dataset/train/17_A047-4463_153D+-+2017-05-11+09.40.22_TissueTrain_(1.00,30774,15012,3897,4556)/17_A047-4463_153D+-+2017-05-11+09.40.22_TissueTrain_(1.00,30774,15012,3897,4556)_img.png')
epi_contours, _ = aida_ann.get_layer_points('epithelium', contour_format=True)
cv2.drawContours(large_image, epi_contours, -1, (0, 0, 0), 30)
lumen_contours, _ = aida_ann.get_layer_points('lumen', contour_format=True)
cv2.drawContours(large_image, lumen_contours, -1, (0, 250, 0), 30)
plt.figure(figsize = (10,20))
plt.imshow(large_image[2000:, 0:2800])
print(len(epi_contours), len(lumen_contours))

aida_ann.merge_overlapping_segments(dissimilarity_thresh=5.0, max_iter=3)

# Plot contours after merging
large_image = imageio.imread('/Volumes/A-CH-EXDISK/Projects/Dataset/train/17_A047-4463_153D+-+2017-05-11+09.40.22_TissueTrain_(1.00,30774,15012,3897,4556)/17_A047-4463_153D+-+2017-05-11+09.40.22_TissueTrain_(1.00,30774,15012,3897,4556)_img.png')
epi_contours, _ = aida_ann.get_layer_points('epithelium', contour_format=True)
cv2.drawContours(large_image, epi_contours, -1, (0, 0, 0), 30)
lumen_contours, _ = aida_ann.get_layer_points('lumen', contour_format=True)
cv2.drawContours(large_image, lumen_contours, -1, (0, 250, 0), 30)
plt.figure(figsize = (10,20))
plt.imshow(large_image[2000:, 0:2800])
print(len(epi_contours), len(lumen_contours))

# debug on toy data
import numpy as np
import skimage.draw as draw
from itertools import product
from collections import OrderedDict

# overwrite converter and annotation
value_hier = (0, 1)
test_label_value_map = {
                'up': 1,
                'background': 0
            }
test_label_interval_map = {
                'up': (0.1, 1),
                'background': (0, 0.1)
            }
converter = AnnotationConverter(value_hier=value_hier, 
                                label_value_map=test_label_value_map, 
                                label_interval_map=test_label_interval_map)
aida_ann = AIDAnnotation('test', 'test', ['up'])

mask = np.zeros((500, 1000))
rr, cc = draw.ellipse(250, 500, 150, 200)
mask[rr, cc] = 1
plt.imshow(mask)

tile1, tile2 = mask[:, 0:500].astype(np.uint8), mask[:, 501:].astype(np.uint8)
print(tile1.dtype)

contours1, labels1, boxes1 = converter.mask_to_contour(tile1)
cnt_tile1 = mask.copy()
cv2.drawContours(cnt_tile1, contours1, -1, 50, 10)
x, y, w, h = boxes1[0]
cv2.rectangle(cnt_tile1, (x,y),(x+w,y+h), 30, 2)
plt.imshow(cnt_tile1)

contours2, labels2, boxes2 = converter.mask_to_contour(tile2, x_offset=501, y_offset=0)
cnt_tile2 = mask.copy()
cv2.drawContours(cnt_tile2, contours2, -1, 50, 10)
x, y, w, h = boxes2[0]
cv2.rectangle(cnt_tile2, (x,y), (x+w,y+h), 30, 2)
plt.imshow(cnt_tile2)

contours, labels, boxes = contours1 + contours2, labels1 + labels2, boxes1 + boxes2
for contour, label, box in zip(contours, labels, boxes):
    aida_ann.add_item(label, 'path', tile_rect=box)
    contour = contour.squeeze().astype(int).tolist()  # deal with extra dim at pos 1
    aida_ann.add_segments_to_last_item(contour)

len(aida_ann.get_layer_points('up'))

# check that boxes result as touching (partial overlap):
aida_ann.check_relative_rect_positions(boxes[1], boxes[0], eps=0.5)

# check that closest points are found correctly
points0 = tuple(tuple(point.squeeze()) for point in contours1[0])
points1 = tuple(tuple(point.squeeze()) for point in contours2[0])
closest_points, point_dist = aida_ann.find_closest_points(aida_ann.euclidean_dist, points0, points1)
print(closest_points)

# check contour merging
outer_points = aida_ann.get_merged(points0, points1, closest_points)
outer_contour = [np.array(outer_points)[:, np.newaxis, :]]
outer_cnt_tile = mask.copy()
cv2.drawContours(outer_cnt_tile, outer_contour, -1, 50, 10)
plt.imshow(outer_cnt_tile)

# Check contour merging for overlapping contours:
# overwriter converter and annotation
value_hier = (0, 1)
test_label_value_map = {
                'up': 1,
                'background': 0
            }
test_label_interval_map = {
                'up': (0.1, 1),
                'background': (0, 0.1)
            }
converter = AnnotationConverter(value_hier=value_hier, 
                                label_value_map=test_label_value_map, 
                                label_interval_map=test_label_interval_map)
aida_ann = AIDAnnotation('test', 'test', ['up'])

mask = np.zeros((500, 1000))
rr, cc = draw.ellipse(250, 350, 175, 210)
mask[rr, cc] = 1
rr, cc = draw.ellipse(275, 650, 100, 180)
mask[rr, cc] = 1.1
plt.imshow(mask)

tile1, tile2 = mask[0:450, 0:600].astype(np.uint8), mask[50:, 400:].astype(np.uint8)
print(tile1.shape, tile2.shape)

contours1, labels1, boxes1 = converter.mask_to_contour(tile1, x_offset=0, y_offset=0)
cnt_mask = np.zeros((500, 1000))
cv2.drawContours(cnt_mask, contours1, -1, 90, 10)
x, y, w, h = boxes1[0]
cv2.rectangle(cnt_mask, (x,y),(x+w,y+h), 60, 2)
contours2, labels2, boxes2 = converter.mask_to_contour(tile2, x_offset=400, y_offset=50)
cnt_tile2 = mask.copy()
cv2.drawContours(cnt_mask, contours2, -1, 150, 10)
x, y, w, h = boxes2[0]
cv2.rectangle(cnt_mask, (x,y), (x+w,y+h), 250, 2)
plt.imshow(cnt_mask)

contours, labels, boxes = contours1 + contours2, labels1 + labels2, boxes1 + boxes2
for contour, label, box in zip(contours, labels, boxes):
    aida_ann.add_item(label, 'path', tile_rect=box)
    contour = contour.squeeze().astype(int).tolist()  # deal with extra dim at pos 1
    aida_ann.add_segments_to_last_item(contour)

len(aida_ann.get_layer_points('up'))

# check that boxes result as overlapping
aida_ann.check_relative_rect_positions(boxes[0], boxes[1], eps=0.5)

# remove overlapping points
points0 = tuple(tuple(point.squeeze()) for point in contours1[0])
points1 = tuple(tuple(point.squeeze()) for point in contours2[0])
points1_no_overlap = aida_ann.remove_overlapping_points(points0, points1)
tile = np.ones_like(mask)
cv2.drawContours(tile, [np.array(points0)[:,np.newaxis,:]], -1, 100, 3)
cv2.drawContours(tile, [np.array(points1_no_overlap)[:,np.newaxis,:]], -1, 250, 3)
plt.imshow(tile)

# check that closest points are found correctly
closest_points, point_dist = aida_ann.find_closest_points(aida_ann.euclidean_dist, points0, points1_no_overlap)
print(list(closest_points)[0:10])

# check contour merging
outer_points = aida_ann.get_merged(points0, points1_no_overlap, closest_points, positions='vertical')
outer_contour = [np.array(outer_points)[:, np.newaxis, :]]
outer_cnt_tile = mask.copy()
cv2.drawContours(outer_cnt_tile, outer_contour, -1, 50, 10)
plt.imshow(outer_cnt_tile)

len(points0)

len(aida_ann.get_layer_points('up'))

aida_ann.merge_overlapping_segments()

aida_final_points = aida_ann.get_layer_points('up')
final_contour = np.array(aida_final_points)[:, np.newaxis, :]
cnt_tile = mask.copy()
cv2.drawContours(cnt_tile, final_contour, -1, 50, 10)
plt.imshow(cnt_tile)